# Funzione per il calcolo degli Embeddings

Di seguito implementiamo una funzione per calcolare gli embeddings utilizzando Azure OpenAI. Questa funzione accetta in input un testo (o una lista di testi) e restituisce i corrispondenti embeddings utilizzando il modello specificato.

In [ ]:
def compute_embeddings(text_input, client=None, model="text-embedding-ada-002"):
    """
    Calcola gli embeddings per un testo o una lista di testi
    
    Args:
        text_input: Testo o lista di testi di cui calcolare l'embedding
        client: Client Azure OpenAI (se None, viene usato il client di default)
        model: Nome del modello di embedding da utilizzare
        
    Returns:
        Lista di embeddings o singolo embedding a seconda dell'input
    """
    import numpy as np
    
    # Usa il client di default se non specificato
    if client is None:
        # Assicurati che il client sia definito globalmente
        try:
            global client
        except NameError:
            raise ValueError("Client OpenAI non definito. Inizializza prima il client.")
    
    # Normalizza input - assicurandoci che sia sempre una lista
    is_single_input = False
    if isinstance(text_input, str):
        text_input = [text_input]
        is_single_input = True
    
    # Pulisci i testi
    import re
    cleaned_texts = [re.sub(r'\s+', ' ', text.strip()) for text in text_input]
    
    try:
        # Chiama l'API per ottenere gli embeddings
        response = client.embeddings.create(
            input=cleaned_texts,
            model=model
        )
        
        # Estrai gli embeddings dalla risposta
        embeddings = [item.embedding for item in response.data]
        
        # Restituisci un singolo embedding o una lista basandosi sull'input originale
        if is_single_input:
            return embeddings[0]
        else:
            return embeddings
    
    except Exception as e:
        print(f"Errore durante il calcolo degli embeddings: {e}")
        return None

In [ ]:
# Test della funzione compute_embeddings
try:
    # Test con un singolo testo
    test_text = "Questo è un testo di prova per testare la funzione di embedding."
    single_embedding = compute_embeddings(test_text)
    print(f"Dimensione dell'embedding per un singolo testo: {len(single_embedding)}")
    
    # Test con una lista di testi
    test_texts = [
        "Primo testo di esempio",
        "Secondo testo di esempio per embedding"
    ]
    multiple_embeddings = compute_embeddings(test_texts)
    print(f"Numero di embeddings calcolati: {len(multiple_embeddings)}")
    print(f"Dimensione di ciascun embedding: {len(multiple_embeddings[0])}")
    
    # Calcola similarità tra i due testi
    import numpy as np
    def cosine_similarity(a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    similarity = cosine_similarity(multiple_embeddings[0], multiple_embeddings[1])
    print(f"Similarità coseno tra i due testi: {similarity:.4f}")
    
except Exception as e:
    print(f"Errore durante il test: {e}")

# Utilizzo della funzione di embedding nel Pipeline RAG

Vediamo come integrare la funzione di embedding nel nostro sistema RAG. Useremo questa funzione per:

1. Generare embeddings per il documento di input (query)
2. Confrontare questi embeddings con quelli dei nostri chunks per trovare le informazioni più rilevanti
3. Utilizzare le informazioni più rilevanti per arricchire il prompt per il modello linguistico

In [ ]:
# Esempio pratico di utilizzo degli embeddings nel sistema RAG

def rag_pipeline(document_text, chunks_data, top_k=3):
    """
    Pipeline RAG completa: prende un documento, trova chunks rilevanti e genera una risposta
    
    Args:
        document_text: Il testo del documento da analizzare
        chunks_data: Lista di chunks con embeddings
        top_k: Numero di chunks più rilevanti da utilizzare
    
    Returns:
        Risposta generata dal modello
    """
    # 1. Calcola embedding per il documento (usando solo i primi 1000 caratteri come query)
    print("Generazione embedding per il documento di input...")
    query_text = document_text[:1000]  # Usa i primi 1000 caratteri come query
    query_embedding = compute_embeddings(query_text)
    
    # 2. Trova i chunks più rilevanti in base alla similarità
    print("Ricerca chunks rilevanti...")
    relevant_chunks = get_relevant_chunks(query_embedding, chunks_data, top_k=top_k)
    
    # 3. Estrai il contenuto dei chunk rilevanti
    context = "\n\n".join([chunk['content'] for chunk in relevant_chunks])
    print(f"Trovati {len(relevant_chunks)} chunks rilevanti")
    
    # 4. Crea il prompt RAG-enhanced
    prompt = RAG_ENHANCED_PROMPT.format(context=context, documento=document_text)
    
    # 5. Chiama l'API con il prompt arricchito
    print("Generazione risposta con Azure OpenAI...")
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "Sei un analizzatore testi professionista. In base alle mail che trovi nel testo che ti passo, fornisci come prima risposta un riepilogo sulle richieste dei clienti. E poi formula una risposta per ogni cliente sulla base della sua richiesta."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=1024,
        temperature=1,
    )
    
    # 6. Restituisci la risposta
    return response.choices[0].message.content

# Esempio di utilizzo
# Per testare con un singolo documento:
"""
import os

# Carica un documento di esempio
file = "email1.txt"  # Scegli uno dei file nella cartella documents
with open(os.path.join("documents", file), "r", encoding="utf-8") as f:
    document_text = f.read()

# Esegui il pipeline RAG
response = rag_pipeline(document_text, chunks_data, top_k=3)
print("\nRisposta dalla pipeline RAG:")
print(response)
"""

# Nota sull'utilizzo della funzione di embedding

La funzione `compute_embeddings` può essere utilizzata per sostituire le chiamate dirette al client Azure OpenAI. Per esempio, nel loop di elaborazione dei documenti, invece di scrivere:

```python
query_response = client.embeddings.create(
    input=document_text[:1000],
    model="text-embedding-ada-002"
)
query_embedding = query_response.data[0].embedding
```

È possibile utilizzare semplicemente:

```python
query_embedding = compute_embeddings(document_text[:1000])
```

Questo approccio ha diversi vantaggi:
1. Codice più pulito e leggibile
2. Gestione centralizzata degli errori
3. Facilità di modificare i parametri o il comportamento della funzione in un solo punto
4. Supporto per input singoli o multipli con la stessa interfaccia

## Pipeline NER e anonimizzazione

In [ ]:
import os
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from dotenv import load_dotenv
import openai

import os
import certifi
from huggingface_hub import hf_hub_download, HfApi

# Forza HuggingFace Hub a usare il certificato custom (Zscaler incluso)
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()
os.environ['CURL_CA_BUNDLE'] = certifi.where()

from transformers import pipeline

ner_pipeline = pipeline("ner", 
                   model="Davlan/bert-base-multilingual-cased-ner-hrl",
                   aggregation_strategy="simple"
                   )
# Path to your local model folder
#model_path = r"C:\desktopnodrive\ai-academy\xlm-roberta-base-ner-hrl"
 
# Load tokenizer and model
#tokenizer =AutoTokenizer.from_pretrained(model_path, use_fast=False)
#model = AutoModelForTokenClassification.from_pretrained(model_path)
 
# Create the pipeline
#ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")       
 
# Load environment variables
load_dotenv()
 
# Chat completion model credentials
azure_chat_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_chat_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_chat_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
 
client = openai.AzureOpenAI(
    api_key=azure_chat_api_key,
    azure_endpoint=azure_chat_endpoint,
    api_version=azure_chat_api_version,
    )
DOCUMENT_TYPE_PROMPT = """
Ti fornirò un documento aziendale.
 
Il tuo compito è determinare il tipo di documento, scegliendo tra i seguenti: 
- Mail
- Nota di credito
- Ordine di acquisto
- Contratto
- Altro
 
Devi basarti solo sul contenuto del documento fornito.
 
Ora incollerò il contenuto del documento tra tripli apici. Rispondi semplicemente con il tipo, nulla di più.
 
Documento:
'''
{documento}
'''
"""
 
 
def anonymize_documents():
    os.makedirs("anonymized", exist_ok=True)
    for file in os.listdir("documents"):
        with open(os.path.join("documents", file), "r", encoding="utf-8") as f:
            text = f.read()
        iban_pattern = r'\b[A-Z]{2}\d{2}[A-Z0-9]{1,30}\b'
        iban_pattern1 = r'\b[A-Z]{2}\d{2}(?:\s?[A-Z0-9]{1,4}){1,7}\b'
        fiscal_code_pattern = r'\b([A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z])\b'
        cell_number_pattern = r'\b(?:\+39[\s\.]?)?3[\d\s\.]{8,12}\b' 
        landline_pattern = r'\b0\d{1,3}([\s\.]?\d{2,4}){1,3}\b'# Italian cell number pattern (e.g., 3XXYYYYYYY or +393XXXXXXXXX)
        email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
        # Get NER results with simple aggregation strategy
        model_entities = ner_pipeline(text)
        # Manually find start and end positions for model results
        for entity in model_entities:
            if 'start' not in entity or entity['start'] is None:
                # Find the entity word in the original text
                entity_text = entity['word']
                # Remove leading ▁ if present (XLM-RoBERTa tokenizer specific)
                clean_text = entity_text.replace('▁', ' ').strip()
                # Find position in original text
                position = text.find(clean_text)
                if position != -1:
                    entity['start'] = position
                    entity['end'] = position + len(clean_text)
                    # Update word to match the text in the document
                    entity['word'] = clean_text
        # Convert model entities to the same format as regex matches
        results = []
        for entity in model_entities:
            if 'start' in entity and entity['start'] is not None:
                results.append({
                    "entity": f"B-{entity['entity_group']}",
                    "score": entity['score'],
                    "index": -1,
                    "word": entity['word'],
                    "start": entity['start'],
                    "end": entity['end']
                })
        # Add IBAN matches as NER-like dicts
        iban_matches = []
        for match in re.finditer(iban_pattern, text):
            iban_matches.append({
                "entity": "B-IBAN",
                "score": 1.0,
                "index": -1,
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        # Add IBAN matches as NER-like dicts
        iban_matches1 = []
        for match in re.finditer(iban_pattern1, text):
            iban_matches1.append({
                "entity": "B-IBAN",
                "score": 1.0,
                "index": -1,
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        # Add Fiscal Code matches as NER-like dicts
        fiscal_code_matches = []
        for match in re.finditer(fiscal_code_pattern, text):
            fiscal_code_matches.append({
                "entity": "B-FISCALCODE",
                "score": 1.0,
                "index": -1,
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        # Add Cell Number matches as NER-like dicts
        cell_number_matches = []
        for match in re.finditer(cell_number_pattern, text):
            cell_number_matches.append({
                "entity": "B-CELLNUMBER",
                "score": 1.0,
                "index": -1,
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        # Add Cell Number matches as NER-like dicts
        phone_number_matches = []
        for match in re.finditer(landline_pattern, text):
            phone_number_matches.append({
                "entity": "B-CELLNUMBER",
                "score": 1.0,
                "index": -1,
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        # Add Email Address matches as NER-like dicts
        email_matches = []
        for match in re.finditer(email_pattern, text):
            email_matches.append({
                "entity": "B-EMAIL",
                "score": 1.0,
                "index": -1,
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        # Insert regex matches at the beginning
        results = iban_matches + iban_matches1 + fiscal_code_matches + cell_number_matches + phone_number_matches + email_matches + results
        # Reconstruct entities
        entities = []
        current = None
        for ent in results:
            if ent["entity"].startswith("B-"):
                if current:
                    entities.append(current)
                current = {
                    "entity": ent["entity"][2:],
                    "tokens": [ent["word"]],
                    "start": ent["start"],
                    "end": ent["end"]
                }
            elif ent["entity"].startswith("I-") and current:
                current["tokens"].append(ent["word"])
                current["end"] = ent["end"]
        if current:
            entities.append(current)
        # Map for anonymization
        label_map = {
            "EMAIL": "Email",
            "PER": "Nome",
            "ORG": "Azienda",
            "LOC": "Luogo",
            "IBAN": "IBAN",
            "FISCALCODE": "FISCALCODE",
            "CELLNUMBER": "Cellulare",
        }
        entity_counters = {}
        replacements = []
        for ent in entities:
            label = ent["entity"]
            mapped = label_map.get(label, label)
            entity_counters[mapped] = entity_counters.get(mapped, 0) + 1
 
            # Custom anonymization for IBAN and FISCALCODE
            entity_text = " ".join(ent["tokens"]).strip()
            if label == "IBAN":
                # Mask all but first 4 chars
                masked = entity_text[:4] + "*" * (len(entity_text) - 4)
                placeholder = f"[{masked}]"
            elif label == "FISCALCODE":
                # Mask all but first 3 chars
                masked = entity_text[:3] + "*" * (len(entity_text) - 3) 
                placeholder = f"[{masked}]"
            elif label == "CELLNUMBER":
                masked = entity_text[:3] + "*" * (len(entity_text) - 3)
                placeholder = f"[{masked}]"
            else:
                placeholder = f"[{mapped}_{entity_counters[mapped]}]"
 
            replacements.append((entity_text, placeholder, ent))
        # Sort replacements by start position in reverse order (to avoid position shifts)
        replacements.sort(key=lambda x: x[2]["start"], reverse=True)
        # Replace entities in text
        anonymized_chars = list(text)
        for entity_text, placeholder, entity in replacements:
            start = entity["start"]
            end = entity["end"]
            # Replace the characters at the specified positions
            anonymized_chars[start:end] = placeholder
        anonymized_text = ''.join(anonymized_chars)
        # Write anonymized file
        with open(os.path.join("anonymized", file), "w", encoding="utf-8") as f:
            f.write(anonymized_text)
 
def get_chat_response(prompt):
 
    response = client.chat.completions.create(
    model=azure_chat_deployment,
    messages=[
        {"role": "system", "content": "Sei un assistente AI."},
        {"role": "user", "content": prompt}
    ],
    max_completion_tokens=256,
    temperature=1,
    )
    return response.choices[0].message.content.strip()
 
def test_documents():
    anonymize_documents()
    for file in os.listdir("anonymized"):
        with open(os.path.join("anonymized", file), "r", encoding="utf-8") as f:
            text = f.read()
        prompt = DOCUMENT_TYPE_PROMPT.format(documento=text)
        response = get_chat_response(prompt)
        print(f"[{file}] → Tipo rilevato: {response}")
 
if __name__ == "__main__":
    test_documents()
 

Device set to use cpu


[email1.txt] → Tipo rilevato: Mail
[email10.txt] → Tipo rilevato: Mail
[email2.txt] → Tipo rilevato: Mail
[email3.txt] → Tipo rilevato: Mail
[email4.txt] → Tipo rilevato: Mail
[email5.txt] → Tipo rilevato: Mail
[email6.txt] → Tipo rilevato: Mail
[email7.txt] → Tipo rilevato: Mail
[email8.txt] → Tipo rilevato: Mail
[email9.txt] → Tipo rilevato: Mail


## Embeddings

In [ ]:
import os
import json
import pickle
from typing import List, Dict, Any
from dotenv import load_dotenv
import numpy as np
from openai import AzureOpenAI
import re
from dataclasses import dataclass
from datetime import datetime

# Gestione import PyMuPDF con conflitti
try:
    import pymupdf as fitz
except ImportError:
    try:
        import fitz
    except ImportError:
        # Fallback usando PyPDF2 se PyMuPDF non funziona
        try:
            import PyPDF2
            PYMUPDF_AVAILABLE = False
        except ImportError:
            raise ImportError("Nessuna libreria PDF disponibile. Installa PyMuPDF o PyPDF2")
    else:
        PYMUPDF_AVAILABLE = True
else:
    PYMUPDF_AVAILABLE = True

# Carica le variabili d'ambiente
load_dotenv()

@dataclass
class DocumentChunk:
    """Rappresenta un chunk del documento con i suoi metadati"""
    id: str
    content: str
    page_number: int
    chunk_index: int
    section: str
    subsection: str
    document_type: str
    embedding: List[float] = None
    metadata: Dict[str, Any] = None

class PDFEmbeddingProcessor:
    def __init__(self):
        """Inizializza il processore con il client Azure OpenAI"""
        # Debug: stampa le variabili caricate
        print("Caricamento variabili d'ambiente...")
        
        self.endpoint = os.getenv("ADA_ENDPOINT")
        print(f"ADA_ENDPOINT: {self.endpoint}")
        if not self.endpoint:
            raise ValueError("Devi definire ADA_ENDPOINT nel file .env")
        
        self.api_key = os.getenv("ADA_API_KEY")
        print(f"ADA_API_KEY: {'*' * len(self.api_key) if self.api_key else 'None'}")
        if not self.api_key:
            raise ValueError("Devi definire ADA_API_KEY nel file .env")
        
        self.api_version = os.getenv("ADA_API_VERSION", "2024-02-01")
        self.deployment_name = os.getenv("ADA_DEPLOYMENT_NAME", "text-embedding-ada-002")
        
        # Inizializza il client Azure OpenAI
        self.client = AzureOpenAI(
            azure_endpoint=self.endpoint,
            api_key=self.api_key,
            api_version=self.api_version
        )
        
        # Configurazione chunking
        self.max_chunk_size = 1500  # Caratteri per chunk
        self.overlap_size = 200     # Overlap tra chunks
        
        # Pattern per identificare sezioni nel documento normativo
        self.section_patterns = {
            'regulation': r'Reg\.to \(.*?\)',
            'directive': r'Direttiva \d+/\d+/[A-Z]+',
            'article': r'art\.\s*\d+|articolo\s*\d+',
            'decree': r'Decreto.*?\d{4}',
            'chapter': r'Capo\s+[IVX]+|Titolo\s+[IVX]+',
            'definition': r'Definizioni:|definizioni:',
            'requirements': r'Requisiti|Obblighi',
            'procedure': r'Procedur[ae]|Modalità'
        }

    def extract_text_from_pdf(self, pdf_path: str) -> List[Dict[str, Any]]:
        """Estrae il testo dal PDF mantenendo informazioni sulla pagina"""
        pages_content = []
        
        if PYMUPDF_AVAILABLE:
            return self._extract_with_pymupdf(pdf_path)
        else:
            return self._extract_with_pypdf2(pdf_path)
    
    def _extract_with_pymupdf(self, pdf_path: str) -> List[Dict[str, Any]]:
        """Estrae testo usando PyMuPDF"""
        pages_content = []
        
        try:
            doc = fitz.open(pdf_path)
            
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                text = page.get_text()
                
                if text.strip():  # Solo pagine con contenuto
                    pages_content.append({
                        'page_number': page_num + 1,
                        'content': text,
                        'char_count': len(text)
                    })
            
            doc.close()
            print(f"Estratto testo da {len(pages_content)} pagine con PyMuPDF")
            return pages_content
            
        except Exception as e:
            print(f"Errore nell'estrazione del PDF con PyMuPDF: {e}")
            return []
    
    def _extract_with_pypdf2(self, pdf_path: str) -> List[Dict[str, Any]]:
        """Estrae testo usando PyPDF2 come fallback"""
        pages_content = []
        
        try:
            import PyPDF2
            
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                for page_num, page in enumerate(pdf_reader.pages):
                    text = page.extract_text()
                    
                    if text.strip():  # Solo pagine con contenuto
                        pages_content.append({
                            'page_number': page_num + 1,
                            'content': text,
                            'char_count': len(text)
                        })
            
            print(f"Estratto testo da {len(pages_content)} pagine con PyPDF2")
            return pages_content
            
        except Exception as e:
            print(f"Errore nell'estrazione del PDF con PyPDF2: {e}")
            return []

    def identify_section(self, text: str) -> tuple:
        """Identifica la sezione e sottosezione del testo"""
        text_lower = text.lower()
        
        # Identifica il tipo di sezione principale
        section = "general"
        for section_type, pattern in self.section_patterns.items():
            if re.search(pattern, text, re.IGNORECASE):
                section = section_type
                break
        
        # Estrae sottosezione specifica
        subsection = ""
        
        # Per regolamenti e direttive
        reg_match = re.search(r'(Reg\.to.*?\d{4}/\d+|Direttiva.*?\d{4}/\d+)', text, re.IGNORECASE)
        if reg_match:
            subsection = reg_match.group(1)
        
        # Per articoli
        art_match = re.search(r'(art\.\s*\d+[a-z]*|articolo\s*\d+[a-z]*)', text, re.IGNORECASE)
        if art_match:
            subsection = art_match.group(1)
        
        # Per definizioni specifiche
        if 'definizioni' in text_lower:
            def_match = re.search(r'([a-zA-Z\s]+):', text)
            if def_match:
                subsection = f"def_{def_match.group(1).strip()}"
        
        return section, subsection

    def smart_chunk_text(self, text: str, page_number: int) -> List[str]:
        """Divide il testo in chunks intelligenti basati sulla struttura del documento"""
        chunks = []
        
        # Dividi per paragrafi principali
        paragraphs = re.split(r'\n\s*\n', text)
        
        current_chunk = ""
        
        for paragraph in paragraphs:
            paragraph = paragraph.strip()
            if not paragraph:
                continue
            
            # Se il paragrafo da solo è troppo lungo, dividilo
            if len(paragraph) > self.max_chunk_size:
                # Salva il chunk corrente se non vuoto
                if current_chunk:
                    chunks.append(current_chunk.strip())
                    current_chunk = ""
                
                # Dividi il paragrafo lungo per frasi
                sentences = re.split(r'[.!?]+', paragraph)
                temp_chunk = ""
                
                for sentence in sentences:
                    sentence = sentence.strip()
                    if not sentence:
                        continue
                    
                    if len(temp_chunk + sentence) > self.max_chunk_size:
                        if temp_chunk:
                            chunks.append(temp_chunk.strip())
                        temp_chunk = sentence
                    else:
                        temp_chunk += sentence + ". "
                
                if temp_chunk:
                    current_chunk = temp_chunk
            
            # Se aggiungere questo paragrafo supera la dimensione massima
            elif len(current_chunk + paragraph) > self.max_chunk_size:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = paragraph
            else:
                current_chunk += "\n\n" + paragraph if current_chunk else paragraph
        
        # Aggiungi l'ultimo chunk
        if current_chunk:
            chunks.append(current_chunk.strip())
        
        return chunks

    def create_chunks(self, pages_content: List[Dict[str, Any]]) -> List[DocumentChunk]:
        """Crea chunks dal contenuto delle pagine"""
        all_chunks = []
        chunk_id_counter = 0
        
        for page_data in pages_content:
            page_number = page_data['page_number']
            content = page_data['content']
            
            # Dividi il contenuto della pagina in chunks
            text_chunks = self.smart_chunk_text(content, page_number)
            
            for chunk_index, chunk_text in enumerate(text_chunks):
                if len(chunk_text.strip()) < 50:  # Salta chunks troppo piccoli
                    continue
                
                # Identifica sezione e sottosezione
                section, subsection = self.identify_section(chunk_text)
                
                # Determina il tipo di documento
                doc_type = "normativa"
                if any(keyword in chunk_text.lower() for keyword in ['regolamento', 'reg.to']):
                    doc_type = "regolamento"
                elif any(keyword in chunk_text.lower() for keyword in ['direttiva']):
                    doc_type = "direttiva"
                elif any(keyword in chunk_text.lower() for keyword in ['decreto']):
                    doc_type = "decreto"
                
                # Crea metadata
                metadata = {
                    'char_count': len(chunk_text),
                    'word_count': len(chunk_text.split()),
                    'contains_definitions': 'definizioni' in chunk_text.lower(),
                    'contains_requirements': any(req in chunk_text.lower() for req in ['requisiti', 'obblighi', 'deve', 'devono']),
                    'contains_procedures': any(proc in chunk_text.lower() for proc in ['procedura', 'modalità', 'metodo']),
                    'extraction_timestamp': datetime.now().isoformat()
                }
                
                chunk = DocumentChunk(
                    id=f"chunk_{chunk_id_counter:06d}",
                    content=chunk_text,
                    page_number=page_number,
                    chunk_index=chunk_index,
                    section=section,
                    subsection=subsection,
                    document_type=doc_type,
                    metadata=metadata
                )
                
                all_chunks.append(chunk)
                chunk_id_counter += 1
        
        print(f"Creati {len(all_chunks)} chunks dal documento")
        return all_chunks

    def get_embedding(self, text: str) -> List[float]:
        """Ottiene l'embedding per il testo usando Azure OpenAI"""
        try:
            # Pulisce il testo
            cleaned_text = re.sub(r'\s+', ' ', text.strip())
            
            response = self.client.embeddings.create(
                input=cleaned_text,
                model=self.deployment_name
            )
            
            return response.data[0].embedding
            
        except Exception as e:
            print(f"Errore nel creare embedding: {e}")
            return None

    def create_embeddings(self, chunks: List[DocumentChunk], batch_size: int = 10) -> List[DocumentChunk]:
        """Crea embeddings per tutti i chunks"""
        print("Creazione embeddings in corso...")
        
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i + batch_size]
            
            for chunk in batch:
                embedding = self.get_embedding(chunk.content)
                if embedding:
                    chunk.embedding = embedding
                else:
                    print(f"Fallito embedding per chunk {chunk.id}")
            
            # Progress update
            progress = min(i + batch_size, len(chunks))
            print(f"Processati {progress}/{len(chunks)} chunks ({progress/len(chunks)*100:.1f}%)")
        
        # Filtra chunks senza embedding
        chunks_with_embeddings = [chunk for chunk in chunks if chunk.embedding is not None]
        print(f"Completati embeddings per {len(chunks_with_embeddings)}/{len(chunks)} chunks")
        
        return chunks_with_embeddings

    def save_chunks(self, chunks: List[DocumentChunk], output_path: str):
        """Salva i chunks con embeddings"""
        # Converti in formato serializzabile
        chunks_data = []
        for chunk in chunks:
            chunk_dict = {
                'id': chunk.id,
                'content': chunk.content,
                'page_number': chunk.page_number,
                'chunk_index': chunk.chunk_index,
                'section': chunk.section,
                'subsection': chunk.subsection,
                'document_type': chunk.document_type,
                'embedding': chunk.embedding,
                'metadata': chunk.metadata
            }
            chunks_data.append(chunk_dict)
        
        # Salva in formato pickle per mantenere gli embeddings
        with open(output_path, 'wb') as f:
            pickle.dump(chunks_data, f)
        
        # Salva anche un file JSON senza embeddings per ispezione
        json_path = output_path.replace('.pkl', '_metadata.json')
        json_data = []
        for chunk_dict in chunks_data:
            json_chunk = chunk_dict.copy()
            del json_chunk['embedding']  # Rimuovi embedding per JSON
            json_data.append(json_chunk)
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        
        print(f"Chunks salvati in:")
        print(f"  - {output_path} (con embeddings)")
        print(f"  - {json_path} (metadata)")

    def load_chunks(self, file_path: str) -> List[DocumentChunk]:
        """Carica i chunks da file"""
        with open(file_path, 'rb') as f:
            chunks_data = pickle.load(f)
        
        chunks = []
        for data in chunks_data:
            chunk = DocumentChunk(
                id=data['id'],
                content=data['content'],
                page_number=data['page_number'],
                chunk_index=data['chunk_index'],
                section=data['section'],
                subsection=data['subsection'],
                document_type=data['document_type'],
                embedding=data['embedding'],
                metadata=data['metadata']
            )
            chunks.append(chunk)
        
        return chunks

    def get_stats(self, chunks: List[DocumentChunk]) -> Dict[str, Any]:
        """Restituisce statistiche sui chunks"""
        if not chunks:
            return {}
        
        stats = {
            'total_chunks': len(chunks),
            'total_pages': len(set(chunk.page_number for chunk in chunks)),
            'avg_chunk_size': np.mean([len(chunk.content) for chunk in chunks]),
            'median_chunk_size': np.median([len(chunk.content) for chunk in chunks]),
            'sections': {},
            'document_types': {},
            'chunks_with_embeddings': sum(1 for chunk in chunks if chunk.embedding is not None)
        }
        
        # Conta per sezioni
        for chunk in chunks:
            stats['sections'][chunk.section] = stats['sections'].get(chunk.section, 0) + 1
            stats['document_types'][chunk.document_type] = stats['document_types'].get(chunk.document_type, 0) + 1
        
        return stats

def main():
    """Funzione principale per processare il PDF"""
    # Inizializza il processore
    processor = PDFEmbeddingProcessor()
    
    # Percorsi file
    pdf_path = r"C:\Users\XM745EF\OneDrive - EY\Documents\AI Academy\Hackaton-agentic-RAG\Gruppo_8\pdf_documents\Wiki_Conformità_primi_55.pdf"
    output_path = "chunks_with_embeddings.pkl"
    
    if not os.path.exists(pdf_path):
        print(f"File PDF non trovato: {pdf_path}")
        return
    
    print(f"Processando PDF: {pdf_path}")
    
    # Step 1: Estrai testo dal PDF
    pages_content = processor.extract_text_from_pdf(pdf_path)
    if not pages_content:
        print("Nessun contenuto estratto dal PDF")
        return
    
    # Step 2: Crea chunks
    chunks = processor.create_chunks(pages_content)
    if not chunks:
        print("Nessun chunk creato")
        return
    
    # Step 3: Crea embeddings
    chunks_with_embeddings = processor.create_embeddings(chunks)
    
    # Step 4: Salva risultati
    processor.save_chunks(chunks_with_embeddings, output_path)
    
    # Step 5: Mostra statistiche
    stats = processor.get_stats(chunks_with_embeddings)
    print("\n=== STATISTICHE FINALI ===")
    print(f"Chunks totali: {stats['total_chunks']}")
    print(f"Pagine processate: {stats['total_pages']}")
    print(f"Dimensione media chunk: {stats['avg_chunk_size']:.0f} caratteri")
    print(f"Chunks con embeddings: {stats['chunks_with_embeddings']}")
    print(f"Sezioni identificate: {list(stats['sections'].keys())}")
    print(f"Tipi documento: {list(stats['document_types'].keys())}")

if __name__ == "__main__":
    main()

Caricamento variabili d'ambiente...
ADA_ENDPOINT: https://fedocana.openai.azure.com/
ADA_API_KEY: ************************************************************************************
Processando PDF: C:\Users\XM745EF\OneDrive - EY\Documents\AI Academy\Hackaton-agentic-RAG\Gruppo_8\pdf_documents\Wiki_Conformità_primi_50.pdf
Estratto testo da 50 pagine con PyMuPDF
Creati 201 chunks dal documento
Creazione embeddings in corso...
Processati 10/201 chunks (5.0%)
Processati 20/201 chunks (10.0%)
Processati 30/201 chunks (14.9%)
Processati 40/201 chunks (19.9%)
Processati 50/201 chunks (24.9%)
Processati 60/201 chunks (29.9%)
Processati 70/201 chunks (34.8%)
Processati 80/201 chunks (39.8%)
Processati 90/201 chunks (44.8%)
Processati 100/201 chunks (49.8%)
Processati 110/201 chunks (54.7%)
Processati 120/201 chunks (59.7%)
Processati 130/201 chunks (64.7%)
Processati 140/201 chunks (69.7%)
Processati 150/201 chunks (74.6%)
Processati 160/201 chunks (79.6%)
Processati 170/201 chunks (84.6%)
Pr

## Chiamata OpenAI

In [3]:
import openai
from dotenv import load_dotenv
import os
import pickle
import numpy as np
from typing import List, Dict, Any

# Carica le variabili d'ambiente dal file .env
load_dotenv()

# Leggi le chiavi dal file .env
api_key = os.getenv('OPENAI_API_KEY')
azure_endpoint = os.getenv('AZURE_ENDPOINT')
api_version = os.getenv('API_VERSION')

# Definizione del prompt base
BASE_PROMPT = """
{documento}
"""

# Prompt arricchito con RAG
RAG_ENHANCED_PROMPT = """
Ecco alcune informazioni rilevanti dal nostro database di conoscenza:

{context}

Ora, analizza il seguente documento:

{documento}
"""

# Crea il client Azure OpenAI
client = openai.AzureOpenAI(
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# Funzione per calcolare la similarità coseno
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Funzione per recuperare chunk rilevanti in base alla similarità
def get_relevant_chunks(query_embedding, chunks, top_k=3):
    similarities = []
    for chunk in chunks:
        if chunk['embedding'] is not None:
            similarity = cosine_similarity(query_embedding, chunk['embedding'])
            similarities.append((chunk, similarity))
    
    # Ordina per similarità decrescente
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Prendi i top_k risultati più rilevanti
    return [chunk for chunk, _ in similarities[:top_k]]

# Carica i chunk con embedding
print("Caricamento dei chunk con embeddings...")
try:
    with open("chunks_with_embeddings.pkl", 'rb') as f:
        chunks_data = pickle.load(f)
    print(f"Caricati {len(chunks_data)} chunk con embeddings")
except Exception as e:
    print(f"Errore nel caricamento dei chunk: {e}")
    chunks_data = []

# Testo di esempio da analizzare 
for file in os.listdir("documents"):
    with open(os.path.join("documents", file), "r", encoding="utf-8") as f:
        document_text = f.read()
    
    # Se abbiamo chunks con embeddings, miglioriamo il prompt con RAG
    if chunks_data:
        # Genera embedding per il documento corrente
        print("Creazione embedding per il documento corrente...")
        query_response = client.embeddings.create(
            input=document_text[:1000],  # Usa i primi 1000 caratteri come query
            model="text-embedding-ada-002"  # Assicurati che questo sia il nome corretto del modello
        )
        query_embedding = query_response.data[0].embedding
        
        # Trova i chunk più rilevanti
        relevant_chunks = get_relevant_chunks(query_embedding, chunks_data)
        
        # Estrai il contenuto dei chunk rilevanti
        context = "\n\n".join([chunk['content'] for chunk in relevant_chunks])
        
        # Crea il prompt RAG-enhanced
        prompt = RAG_ENHANCED_PROMPT.format(context=context, documento=document_text)
        print(f"Prompt arricchito con {len(relevant_chunks)} chunk rilevanti")
    else:
        # Usa il prompt base se non abbiamo chunks
        prompt = BASE_PROMPT.format(documento=document_text)
        print("Usando prompt base (senza RAG)")

    # Chiama l'API con il prompt
    print("Chiamata all'API di Azure OpenAI...")
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "Sei un analizzatore testi professionista. In base alle mail che trovi nel testo che ti passo, fornisci come prima risposta un riepilogo sulle richieste dei clienti. E poi formula una risposta per ogni cliente sulla base della sua richiesta."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=1024,
        temperature=1,
    )

    print("\nRisposta dall'AI:")
    print(response.choices[0].message.content)

Caricamento dei chunk con embeddings...
Caricati 201 chunk con embeddings
Creazione embedding per il documento corrente...


BadRequestError: Unsupported data type

## Utility RAG: Funzioni per valutare l'efficacia del sistema RAG

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import time
import json
from typing import List, Dict, Any, Tuple
import numpy as np

class RAGEvaluator:
    """Classe per valutare e visualizzare l'efficacia del sistema RAG"""
    
    def __init__(self, chunks_data=None):
        """Inizializza l'evaluator con i dati dei chunk"""
        self.chunks_data = chunks_data
        if chunks_data is None:
            try:
                with open("chunks_with_embeddings.pkl", 'rb') as f:
                    self.chunks_data = pickle.load(f)
                print(f"Caricati {len(self.chunks_data)} chunk con embeddings")
            except Exception as e:
                print(f"Errore nel caricamento dei chunk: {e}")
                self.chunks_data = []
    
    def get_embedding(self, text: str, client=None) -> List[float]:
        """Ottiene l'embedding per un testo"""
        if client is None:
            # Assumi che client sia globale
            from openai import AzureOpenAI
            load_dotenv()
            client = AzureOpenAI(
                api_key=os.getenv('ADA_API_KEY'),
                azure_endpoint=os.getenv('ADA_ENDPOINT'),
                api_version=os.getenv('ADA_API_VERSION', "2024-02-01")
            )
        
        try:
            response = client.embeddings.create(
                input=text,
                model="text-embedding-ada-002"
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Errore nell'ottenere embedding: {e}")
            return None
    
    def find_similar_chunks(self, query: str, top_k: int = 5, client=None) -> List[Dict]:
        """Trova i chunk più simili a una query"""
        # Ottieni embedding per la query
        query_embedding = self.get_embedding(query, client)
        if query_embedding is None:
            return []
        
        # Calcola similarità con tutti i chunk
        similarities = []
        for chunk in self.chunks_data:
            if chunk['embedding'] is not None:
                similarity = self.cosine_similarity(query_embedding, chunk['embedding'])
                similarities.append((chunk, similarity))
        
        # Ordina per similarità decrescente e prendi i top_k
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Aggiungi il punteggio di similarità ai chunk
        result = []
        for chunk, score in similarities[:top_k]:
            chunk_with_score = chunk.copy()
            chunk_with_score['similarity_score'] = score
            result.append(chunk_with_score)
        
        return result
    
    @staticmethod
    def cosine_similarity(a: List[float], b: List[float]) -> float:
        """Calcola la similarità coseno tra due vettori"""
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    def visualize_embeddings_2d(self, query=None, query_embedding=None, top_n: int = 100):
        """Visualizza gli embeddings in 2D usando t-SNE"""
        if not self.chunks_data:
            print("Nessun chunk disponibile per la visualizzazione")
            return
        
        # Limita il numero di chunk per la visualizzazione
        sample_size = min(top_n, len(self.chunks_data))
        sampled_chunks = self.chunks_data[:sample_size]
        
        # Estrai gli embedding
        embeddings = [chunk['embedding'] for chunk in sampled_chunks]
        
        # Aggiungi query embedding se fornito
        if query_embedding is not None:
            embeddings.append(query_embedding)
        elif query is not None:
            query_emb = self.get_embedding(query)
            if query_emb:
                embeddings.append(query_emb)
        
        # Applica t-SNE
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(embeddings)-1))
        reduced_embeddings = tsne.fit_transform(embeddings)
        
        # Prepara per la visualizzazione
        plt.figure(figsize=(10, 8))
        
        # Distingui tra query e altri chunk
        query_point = None
        if query_embedding is not None or query is not None:
            query_point = reduced_embeddings[-1]
            chunk_points = reduced_embeddings[:-1]
        else:
            chunk_points = reduced_embeddings
        
        # Visualizza i chunk
        plt.scatter(
            chunk_points[:, 0], 
            chunk_points[:, 1], 
            alpha=0.6, 
            c=[chunk.get('section_color', 'blue') for chunk in sampled_chunks]
        )
        
        # Visualizza la query se presente
        if query_point is not None:
            plt.scatter(
                query_point[0], 
                query_point[1], 
                marker='*', 
                s=200, 
                c='red', 
                label='Query'
            )
        
        plt.title('t-SNE Visualization of Document Embeddings')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()
    
    def enhance_prompt_with_rag(self, documento: str, top_k: int = 3, client=None) -> Tuple[str, List[Dict]]:
        """Arricchisce un prompt con contenuto RAG rilevante"""
        # Template per il prompt RAG
        rag_template = """
Ecco alcune informazioni rilevanti dal nostro database di conoscenza:

{context}

Ora, analizza il seguente documento:

{documento}
"""
        
        # Ottieni l'embedding per il documento
        query_embedding = self.get_embedding(documento[:1000], client)  # Usa i primi 1000 caratteri
        if query_embedding is None:
            return documento, []
        
        # Trova chunk rilevanti
        relevant_chunks = self.find_similar_chunks(documento[:1000], top_k, client)
        
        # Se non abbiamo trovato chunk rilevanti, restituisci il documento originale
        if not relevant_chunks:
            return documento, []
        
        # Estrai contenuto dai chunk rilevanti
        context_parts = []
        for i, chunk in enumerate(relevant_chunks):
            context_parts.append(f"[Fonte {i+1}] (Similarità: {chunk['similarity_score']:.4f})\n{chunk['content']}")
        
        context = "\n\n".join(context_parts)
        
        # Crea il prompt RAG-enhanced
        enhanced_prompt = rag_template.format(context=context, documento=documento)
        
        return enhanced_prompt, relevant_chunks
    
    def evaluate_rag_effectiveness(self, queries: List[str], top_k_values: List[int] = [1, 3, 5, 10]):
        """Valuta l'efficacia del RAG per diverse query e configurazioni"""
        results = {}
        
        for query in queries:
            query_result = {'query': query, 'top_k_results': {}}
            print(f"Valutando query: '{query[:50]}...'")
            
            for top_k in top_k_values:
                start_time = time.time()
                enhanced_prompt, relevant_chunks = self.enhance_prompt_with_rag(query, top_k)
                end_time = time.time()
                
                query_result['top_k_results'][top_k] = {
                    'time_taken': end_time - start_time,
                    'num_chunks': len(relevant_chunks),
                    'avg_similarity': np.mean([chunk['similarity_score'] for chunk in relevant_chunks]) if relevant_chunks else 0,
                    'relevant_chunk_ids': [chunk['id'] for chunk in relevant_chunks]
                }
                
                print(f"  - Top {top_k}: trovati {len(relevant_chunks)} chunk, "
                     f"similarità media: {query_result['top_k_results'][top_k]['avg_similarity']:.4f}")
            
            results[query[:20]] = query_result
        
        return results
    
    def save_evaluation_results(self, results, filename="rag_evaluation_results.json"):
        """Salva i risultati della valutazione in un file JSON"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        print(f"Risultati salvati in {filename}")

# Demo per testare il sistema RAG
def test_rag_system():
    # Inizializza l'evaluator
    evaluator = RAGEvaluator()
    
    # Esempio di query da testare
    test_query = """
    Sono interessato alle norme sulla protezione dei dati personali e sulla gestione dei dati sensibili nell'ambito del GDPR.
    Potresti fornirmi informazioni sugli obblighi di conformità e sulle possibili sanzioni in caso di violazioni?
    """
    
    # Ottieni e stampa i chunk più simili
    similar_chunks = evaluator.find_similar_chunks(test_query, top_k=3)
    print(f"Trovati {len(similar_chunks)} chunk simili alla query.\n")
    
    for i, chunk in enumerate(similar_chunks):
        print(f"Chunk #{i+1} (ID: {chunk['id']}, Score: {chunk['similarity_score']:.4f}):")
        print(f"Pagina: {chunk['page_number']}, Sezione: {chunk['section']}")
        print(f"Contenuto: {chunk['content'][:150]}...\n")
    
    # Arricchisci il prompt con RAG
    enhanced_prompt, _ = evaluator.enhance_prompt_with_rag(test_query)
    print("Prompt arricchito con RAG:")
    print(enhanced_prompt[:500] + "...\n")
    
    # Visualizza gli embedding
    evaluator.visualize_embeddings_2d(test_query, top_n=50)

# Decommentare per eseguire il test
# test_rag_system()

## Implementazione RAG per DOCUMENT_TYPE_PROMPT

In [ ]:
import os
import pickle
import numpy as np
from dotenv import load_dotenv
import openai
import time

# Carica le variabili d'ambiente dal file .env
load_dotenv()

# Chiavi di Azure OpenAI
azure_chat_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_chat_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_chat_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# Configurazione embedding
azure_embedding_api_key = os.getenv("ADA_API_KEY")
azure_embedding_endpoint = os.getenv("ADA_ENDPOINT")
azure_embedding_api_version = os.getenv("ADA_API_VERSION", "2024-02-01")
azure_embedding_deployment = os.getenv("ADA_DEPLOYMENT_NAME", "text-embedding-ada-002")

# Crea i client di Azure OpenAI
chat_client = openai.AzureOpenAI(
    api_key=azure_chat_api_key,
    azure_endpoint=azure_chat_endpoint,
    api_version=azure_chat_api_version
)

embedding_client = openai.AzureOpenAI(
    api_key=azure_embedding_api_key, 
    azure_endpoint=azure_embedding_endpoint,
    api_version=azure_embedding_api_version
)

# Carica i chunks con embedding
print("Caricamento dei chunk con embeddings...")
try:
    with open("chunks_with_embeddings.pkl", 'rb') as f:
        chunks_data = pickle.load(f)
    print(f"Caricati {len(chunks_data)} chunk con embeddings")
except Exception as e:
    print(f"Errore nel caricamento dei chunk: {e}")
    chunks_data = []

# Definizione dei prompt
# Prompt originale
DOCUMENT_TYPE_PROMPT = """
Ti fornirò un documento aziendale.
 
Il tuo compito è determinare il tipo di documento, scegliendo tra i seguenti: 
- Mail
- Nota di credito
- Ordine di acquisto
- Contratto
- Altro
 
Devi basarti solo sul contenuto del documento fornito.
 
Ora incollerò il contenuto del documento tra tripli apici. Rispondi semplicemente con il tipo, nulla di più.
 
Documento:
'''
{documento}
'''
"""

# Prompt arricchito con RAG
RAG_DOCUMENT_TYPE_PROMPT = """
Ti fornirò un documento aziendale.

Il tuo compito è determinare il tipo di documento, scegliendo tra i seguenti: 
- Mail
- Nota di credito
- Ordine di acquisto
- Contratto
- Altro

Devi basarti principalmente sul contenuto del documento fornito.

Per aiutarti, ecco alcune informazioni rilevanti dal nostro database di conoscenza che potrebbero aiutarti nell'analisi:

{context}

Ora incollerò il contenuto del documento tra tripli apici. Rispondi semplicemente con il tipo, nulla di più.

Documento:
'''
{documento}
'''
"""

# Funzione per calcolare la similarità coseno
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Funzione per recuperare chunk rilevanti
def get_relevant_chunks(query_embedding, chunks_data, top_k=3):
    if not chunks_data:
        return []
    
    similarities = []
    for chunk in chunks_data:
        if chunk['embedding'] is not None:
            similarity = cosine_similarity(query_embedding, chunk['embedding'])
            similarities.append((chunk, similarity))
    
    # Ordina per similarità decrescente
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Prendi i top_k risultati più rilevanti
    return [(chunk, sim) for chunk, sim in similarities[:top_k]]

# Funzione per generare l'embedding di un testo
def get_embedding(text):
    try:
        response = embedding_client.embeddings.create(
            input=text,
            model=azure_embedding_deployment
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Errore nel generare embedding: {e}")
        return None

# Funzione per classificare un documento con RAG
def classify_document_with_rag(document_text, use_rag=True, top_k=3):
    start_time = time.time()
    
    if use_rag and chunks_data:
        # Genera embedding per il documento
        print("Generazione embedding per il documento...")
        doc_embedding = get_embedding(document_text[:1000])  # Usa i primi 1000 caratteri
        
        if doc_embedding:
            # Trova chunk rilevanti
            print(f"Ricerca dei {top_k} chunk più rilevanti...")
            relevant_chunks = get_relevant_chunks(doc_embedding, chunks_data, top_k)
            
            # Estrai contenuto e punteggi
            context_parts = []
            for i, (chunk, similarity) in enumerate(relevant_chunks):
                context_parts.append(
                    f"[FONTE {i+1}] (Similitudine: {similarity:.4f}, Sezione: {chunk.get('section', 'N/A')})\n"
                    f"{chunk['content']}"
                )
            
            context = "\n\n".join(context_parts)
            prompt = RAG_DOCUMENT_TYPE_PROMPT.format(context=context, documento=document_text)
            print(f"Prompt arricchito con {len(relevant_chunks)} fonti dalla knowledge base")
        else:
            print("Impossibile generare embedding, uso il prompt standard")
            prompt = DOCUMENT_TYPE_PROMPT.format(documento=document_text)
    else:
        prompt = DOCUMENT_TYPE_PROMPT.format(documento=document_text)
        print("Usando prompt standard (senza RAG)")
    
    # Chiamata all'API
    print("Chiamata all'API di Azure OpenAI per classificazione...")
    response = chat_client.chat.completions.create(
        model=azure_chat_deployment,
        messages=[
            {"role": "system", "content": "Sei un analista di documenti aziendali esperto."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=256,
        temperature=0.3,  # Temperatura più bassa per risposta più consistente
    )
    
    result = response.choices[0].message.content.strip()
    
    # Calcola il tempo impiegato
    elapsed_time = time.time() - start_time
    print(f"Classificazione completata in {elapsed_time:.2f} secondi")
    
    return result, prompt

# Test di esempio con diversi file nella cartella "anonymized"
def test_document_classification(use_rag=True):
    results = []
    
    # Trova i file nella directory anonymized
    directory = "anonymized"
    if not os.path.isdir(directory):
        print(f"La directory '{directory}' non esiste.")
        return
    
    files = os.listdir(directory)
    for file in files[:3]:  # Limita a 3 file per test
        file_path = os.path.join(directory, file)
        
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                document_text = f.read()
            
            print(f"\nAnalisi del file: {file}")
            document_type, _ = classify_document_with_rag(document_text, use_rag=use_rag)
            
            results.append({
                "file": file,
                "type": document_type
            })
            
            print(f"Tipo documento rilevato: {document_type}")
            
        except Exception as e:
            print(f"Errore nell'elaborazione del file {file}: {e}")
    
    return results

# Esegui il test - commenta/decommenta per eseguire
# test_results = test_document_classification(use_rag=True)

# Esempio di comparazione RAG vs No-RAG
def compare_rag_vs_no_rag():
    print("Comparazione RAG vs No-RAG per la classificazione dei documenti")
    
    # Scegli un file di esempio
    file_path = None
    if os.path.isdir("anonymized") and os.listdir("anonymized"):
        file_path = os.path.join("anonymized", os.listdir("anonymized")[0])
    elif os.path.isdir("documents") and os.listdir("documents"):
        file_path = os.path.join("documents", os.listdir("documents")[0])
    
    if not file_path:
        print("Nessun file trovato per il test")
        return
    
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            document_text = f.read()
        
        print(f"\nAnalisi del file: {os.path.basename(file_path)}")
        
        # Test senza RAG
        print("\n--- CLASSIFICAZIONE SENZA RAG ---")
        no_rag_type, no_rag_prompt = classify_document_with_rag(document_text, use_rag=False)
        
        # Test con RAG
        print("\n--- CLASSIFICAZIONE CON RAG ---")
        rag_type, rag_prompt = classify_document_with_rag(document_text, use_rag=True)
        
        print("\n--- RISULTATI ---")
        print(f"Tipo documento (senza RAG): {no_rag_type}")
        print(f"Tipo documento (con RAG): {rag_type}")
        
        return {
            "file": os.path.basename(file_path),
            "no_rag_type": no_rag_type,
            "rag_type": rag_type,
            "no_rag_prompt_length": len(no_rag_prompt),
            "rag_prompt_length": len(rag_prompt)
        }
        
    except Exception as e:
        print(f"Errore durante il confronto RAG vs No-RAG: {e}")
        return None

# Decommentare per eseguire la comparazione
# comparison_result = compare_rag_vs_no_rag()

## Esempio Pratico: Analisi di Documenti con RAG

In [ ]:
# Imposta this per eseguire l'esempio pratico
run_practical_example = True  # Modifica a True per eseguire

if run_practical_example:
    import pandas as pd
    import matplotlib.pyplot as plt
    from tqdm.notebook import tqdm
    import time
    
    # Configurazione dell'esperimento
    print("Avvio dell'analisi comparativa RAG vs Non-RAG...")
    
    # Ottieni lista dei file da processare
    directory = "anonymized" if os.path.isdir("anonymized") else "documents"
    files = os.listdir(directory)[:5]  # Limita a 5 file per brevità
    
    # Inizializza strutture dati per risultati
    results = []
    
    # Processa ogni file con e senza RAG
    for file in tqdm(files, desc="Analisi documenti"):
        file_path = os.path.join(directory, file)
        
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                document_text = f.read()
            
            # Analisi senza RAG
            start_time = time.time()
            no_rag_type, no_rag_prompt = classify_document_with_rag(document_text, use_rag=False)
            no_rag_time = time.time() - start_time
            
            # Analisi con RAG
            start_time = time.time()
            rag_type, rag_prompt = classify_document_with_rag(document_text, use_rag=True, top_k=3)
            rag_time = time.time() - start_time
            
            # Salva risultati
            results.append({
                "file": file,
                "no_rag_type": no_rag_type,
                "rag_type": rag_type,
                "no_rag_time": no_rag_time,
                "rag_time": rag_time,
                "no_rag_prompt_length": len(no_rag_prompt),
                "rag_prompt_length": len(rag_prompt),
                "match": no_rag_type == rag_type
            })
            
        except Exception as e:
            print(f"Errore nell'analisi di {file}: {e}")
    
    # Crea dataframe con risultati
    df = pd.DataFrame(results)
    
    # Statistiche sui risultati
    print("\n=== RISULTATI DELL'ANALISI COMPARATIVA ===")
    print(f"Documenti analizzati: {len(df)}")
    print(f"Concordanza classificazioni: {df['match'].sum()}/{len(df)} ({df['match'].mean()*100:.1f}%)")
    print(f"Tempo medio senza RAG: {df['no_rag_time'].mean():.2f} secondi")
    print(f"Tempo medio con RAG: {df['rag_time'].mean():.2f} secondi")
    print(f"Overhead medio RAG: {(df['rag_time'].mean() / df['no_rag_time'].mean() - 1) * 100:.1f}%")
    
    # Visualizzazione dei risultati
    plt.figure(figsize=(12, 5))
    
    # Confronto tempi di esecuzione
    plt.subplot(1, 2, 1)
    plt.bar(['Standard', 'RAG'], [df['no_rag_time'].mean(), df['rag_time'].mean()], color=['blue', 'orange'])
    plt.title('Tempo medio di classificazione')
    plt.ylabel('Secondi')
    
    # Confronto lunghezza prompt
    plt.subplot(1, 2, 2)
    plt.bar(['Standard', 'RAG'], [df['no_rag_prompt_length'].mean(), df['rag_prompt_length'].mean()], color=['blue', 'orange'])
    plt.title('Lunghezza media prompt')
    plt.ylabel('Caratteri')
    
    plt.tight_layout()
    plt.show()
    
    # Dettaglio per ogni documento
    print("\n--- CLASSIFICAZIONE PER DOCUMENTO ---")
    for _, row in df.iterrows():
        match_symbol = "✓" if row["match"] else "✗" 
        print(f"{row['file']}: Standard: {row['no_rag_type']} | RAG: {row['rag_type']} {match_symbol}")
else:
    print("Per eseguire l'esempio pratico, imposta 'run_practical_example = True' e riesegui la cella.")

## Implementazione di un Workflow Agentico per RAG

In [ ]:
import os
import json
import pickle
import time
import re
import uuid
from typing import List, Dict, Any, Tuple, Optional, Union
from dataclasses import dataclass, field, asdict
from datetime import datetime
import numpy as np
from dotenv import load_dotenv
import openai

# Carica le variabili d'ambiente
load_dotenv()

# Configurazione di Azure OpenAI
AZURE_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
ADA_API_KEY = os.getenv("ADA_API_KEY")
ADA_ENDPOINT = os.getenv("ADA_ENDPOINT")
ADA_API_VERSION = os.getenv("ADA_API_VERSION", "2024-02-01")
ADA_DEPLOYMENT = os.getenv("ADA_DEPLOYMENT_NAME", "text-embedding-ada-002")

# Inizializza i client di OpenAI
chat_client = openai.AzureOpenAI(
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embedding_client = openai.AzureOpenAI(
    api_key=ADA_API_KEY,
    azure_endpoint=ADA_ENDPOINT,
    api_version=ADA_API_VERSION
)

# Definizione delle strutture dati per l'agente
@dataclass
class Document:
    """Rappresenta un documento da analizzare"""
    id: str
    content: str
    file_path: Optional[str] = None
    file_name: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)
    doc_type: Optional[str] = None
    embedding: Optional[List[float]] = None
    classification_confidence: float = 0.0
    processed_timestamp: Optional[str] = None

@dataclass
class KnowledgeChunk:
    """Rappresenta un chunk di conoscenza dal database"""
    id: str
    content: str
    embedding: List[float]
    metadata: Dict[str, Any] = field(default_factory=dict)
    relevance_score: float = 0.0

@dataclass
class ReasoningStep:
    """Rappresenta uno step di ragionamento dell'agente"""
    id: str
    thought: str
    action: str
    action_input: Dict[str, Any]
    action_output: Any = None
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class AgentState:
    """Rappresenta lo stato corrente dell'agente"""
    agent_id: str
    document: Optional[Document] = None
    relevant_chunks: List[KnowledgeChunk] = field(default_factory=list)
    reasoning_steps: List[ReasoningStep] = field(default_factory=list)
    final_decision: Optional[str] = None
    confidence: float = 0.0
    start_time: str = field(default_factory=lambda: datetime.now().isoformat())
    end_time: Optional[str] = None
    status: str = "initialized"  # initialized, processing, completed, failed
    error: Optional[str] = None

class AgenticRAG:
    """Implementa un sistema RAG con capacità agentiche"""
    
    def __init__(self, knowledge_base_path: str = "chunks_with_embeddings.pkl"):
        """Inizializza il sistema RAG agentico"""
        self.knowledge_chunks = self._load_knowledge_base(knowledge_base_path)
        print(f"Knowledge base caricata: {len(self.knowledge_chunks)} chunk disponibili")
    
    def _load_knowledge_base(self, knowledge_base_path: str) -> List[KnowledgeChunk]:
        """Carica i chunk dalla knowledge base"""
        try:
            with open(knowledge_base_path, 'rb') as f:
                raw_chunks = pickle.load(f)
            
            # Converti in KnowledgeChunk
            knowledge_chunks = []
            for chunk in raw_chunks:
                if chunk.get('embedding') is not None:
                    knowledge_chunks.append(
                        KnowledgeChunk(
                            id=chunk.get('id', str(uuid.uuid4())),
                            content=chunk.get('content', ''),
                            embedding=chunk.get('embedding', []),
                            metadata=chunk.get('metadata', {})
                        )
                    )
            
            return knowledge_chunks
        except Exception as e:
            print(f"Errore nel caricamento della knowledge base: {e}")
            return []
    
    def create_embedding(self, text: str) -> Optional[List[float]]:
        """Genera embedding per un testo"""
        try:
            # Pulisci e tronca il testo se necessario
            cleaned_text = re.sub(r'\s+', ' ', text.strip())
            if len(cleaned_text) > 8000:
                cleaned_text = cleaned_text[:8000]  # Limite per evitare errori API
            
            response = embedding_client.embeddings.create(
                input=cleaned_text,
                model=ADA_DEPLOYMENT
            )
            
            return response.data[0].embedding
        except Exception as e:
            print(f"Errore nella creazione dell'embedding: {e}")
            return None
    
    def cosine_similarity(self, embedding1: List[float], embedding2: List[float]) -> float:
        """Calcola la similarità coseno tra due embedding"""
        return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    
    def retrieve_relevant_chunks(self, document: Document, top_k: int = 5) -> List[KnowledgeChunk]:
        """Recupera i chunk più rilevanti per il documento"""
        if document.embedding is None:
            document.embedding = self.create_embedding(document.content[:1500])
            if document.embedding is None:
                return []
        
        # Calcola similarità con tutti i chunk
        chunk_similarities = []
        for chunk in self.knowledge_chunks:
            similarity = self.cosine_similarity(document.embedding, chunk.embedding)
            chunk_with_score = KnowledgeChunk(
                id=chunk.id,
                content=chunk.content,
                embedding=chunk.embedding,
                metadata=chunk.metadata,
                relevance_score=similarity
            )
            chunk_similarities.append((chunk_with_score, similarity))
        
        # Ordina per similarità decrescente
        chunk_similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Prendi i top_k risultati e aggiorna i punteggi
        top_chunks = [chunk for chunk, _ in chunk_similarities[:top_k]]
        return top_chunks
    
    def _generate_system_prompt(self) -> str:
        """Genera il prompt di sistema per l'agente"""
        return """Sei un assistente AI avanzato che analizza documenti aziendali.
Il tuo compito è determinare il tipo di documento e ragionare in modo chiaro e strutturato.
Per ogni documento devi:
1. Analizzare attentamente il contenuto
2. Considerare le informazioni fornite dalla knowledge base
3. Identificare pattern e caratteristiche chiave
4. Formulare un ragionamento strutturato
5. Prendere una decisione basata sui fatti

Risponderai in formato JSON con i seguenti campi:
- thoughts: il tuo ragionamento dettagliato
- observations: osservazioni rilevanti sul documento
- classification: il tipo di documento (Mail, Nota di credito, Ordine di acquisto, Contratto, Altro)
- confidence: un valore da 0 a 1 che indica la tua sicurezza nella classificazione
- nextAction: l'azione successiva consigliata (es. "extract_info", "request_more_context", "final_decision")
"""
    
    def _generate_user_prompt(self, document: Document, relevant_chunks: List[KnowledgeChunk]) -> str:
        """Genera il prompt utente con il documento e i chunk rilevanti"""
        context_parts = []
        
        # Aggiungi i chunk rilevanti come contesto
        for i, chunk in enumerate(relevant_chunks):
            context_parts.append(
                f"[FONTE {i+1}] (Rilevanza: {chunk.relevance_score:.4f})\n{chunk.content}"
            )
        
        context = "\n\n".join(context_parts) if context_parts else "Nessun contesto rilevante disponibile."
        
        return f"""# CONTESTO DALLA KNOWLEDGE BASE
{context}

# DOCUMENTO DA ANALIZZARE
```
{document.content}
```

Analizza questo documento e determina il suo tipo, scegliendo tra: Mail, Nota di credito, Ordine di acquisto, Contratto, Altro.
Fornisci un ragionamento dettagliato sulla tua classificazione e proponi l'azione successiva appropriata.
"""
    
    def parse_agent_response(self, response_text: str) -> Dict[str, Any]:
        """Analizza la risposta dell'agente e la converte in un dizionario strutturato"""
        # Cerca di trovare JSON nella risposta
        json_match = re.search(r'```json\n(.*?)\n```', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # Cerca JSON senza delimitatori di codice
            json_match = re.search(r'(\{.*\})', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
            else:
                # Fallback: estrai campi chiave dal testo
                return self._extract_fields_from_text(response_text)
        
        # Prova a parsare il JSON
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Errore nel parsing JSON: {e}")
            return self._extract_fields_from_text(response_text)
    
    def _extract_fields_from_text(self, text: str) -> Dict[str, Any]:
        """Estrai campi chiave dal testo quando il parsing JSON fallisce"""
        result = {
            "thoughts": "",
            "observations": "",
            "classification": "Altro",
            "confidence": 0.5,
            "nextAction": "final_decision"
        }
        
        # Estrai classificazione
        classification_match = re.search(r'classificazione:?\s*(Mail|Nota di credito|Ordine di acquisto|Contratto|Altro)', text, re.IGNORECASE)
        if classification_match:
            result["classification"] = classification_match.group(1)
        
        # Estrai confidence se presente
        confidence_match = re.search(r'confidence:?\s*(\d+\.?\d*)', text)
        if confidence_match:
            try:
                result["confidence"] = float(confidence_match.group(1))
            except ValueError:
                pass
        
        # Estrai thoughts
        thoughts_match = re.search(r'thoughts:?\s*(.*?)(?:observations|classification|confidence|nextAction|\Z)', text, re.DOTALL | re.IGNORECASE)
        if thoughts_match:
            result["thoughts"] = thoughts_match.group(1).strip()
        
        return result
    
    def process_document(self, document_content: str, document_id: Optional[str] = None, metadata: Dict[str, Any] = None) -> AgentState:
        """Processa un documento dall'inizio alla fine"""
        agent_id = str(uuid.uuid4())
        agent_state = AgentState(agent_id=agent_id)
        
        try:
            # Crea l'oggetto documento
            document = Document(
                id=document_id or str(uuid.uuid4()),
                content=document_content,
                metadata=metadata or {},
                processed_timestamp=datetime.now().isoformat()
            )
            agent_state.document = document
            agent_state.status = "processing"
            
            # Step 1: Genera embedding per il documento
            print(f"[Agente {agent_id[:8]}] Generazione embedding per il documento...")
            document.embedding = self.create_embedding(document.content[:1500])
            
            if document.embedding is None:
                agent_state.status = "failed"
                agent_state.error = "Impossibile generare embedding per il documento"
                return agent_state
            
            # Step 2: Recupera chunk rilevanti
            print(f"[Agente {agent_id[:8]}] Recupero chunk rilevanti...")
            relevant_chunks = self.retrieve_relevant_chunks(document)
            agent_state.relevant_chunks = relevant_chunks
            
            # Registra lo step di ragionamento
            agent_state.reasoning_steps.append(
                ReasoningStep(
                    id="step_retrieval",
                    thought="Recupero informazioni rilevanti dalla knowledge base",
                    action="retrieve_chunks",
                    action_input={"document_id": document.id},
                    action_output=f"Recuperati {len(relevant_chunks)} chunk rilevanti"
                )
            )
            
            # Step 3: Genera prompt per l'agente
            system_prompt = self._generate_system_prompt()
            user_prompt = self._generate_user_prompt(document, relevant_chunks)
            
            # Step 4: Chiama l'API per il ragionamento e la classificazione
            print(f"[Agente {agent_id[:8]}] Analisi del documento in corso...")
            response = chat_client.chat.completions.create(
                model=AZURE_DEPLOYMENT,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.2,
                response_format={"type": "json_object"}
            )
            
            response_text = response.choices[0].message.content
            parsed_response = self.parse_agent_response(response_text)
            
            # Registra lo step di ragionamento
            agent_state.reasoning_steps.append(
                ReasoningStep(
                    id="step_analysis",
                    thought=parsed_response.get("thoughts", ""),
                    action="analyze_document",
                    action_input={"document_id": document.id},
                    action_output=parsed_response
                )
            )
            
            # Step 5: Prendi la decisione finale
            document.doc_type = parsed_response.get("classification", "Altro")
            document.classification_confidence = parsed_response.get("confidence", 0.5)
            
            agent_state.final_decision = document.doc_type
            agent_state.confidence = document.classification_confidence
            agent_state.status = "completed"
            agent_state.end_time = datetime.now().isoformat()
            
            # Step 6: Se il sistema suggerisce un'azione successiva, registrala
            next_action = parsed_response.get("nextAction")
            if next_action and next_action != "final_decision":
                agent_state.reasoning_steps.append(
                    ReasoningStep(
                        id="step_next_action",
                        thought=f"L'analisi suggerisce un'azione successiva: {next_action}",
                        action=next_action,
                        action_input={"document_id": document.id},
                        action_output=None  # Da eseguire in seguito
                    )
                )
            
            return agent_state
            
        except Exception as e:
            agent_state.status = "failed"
            agent_state.error = str(e)
            agent_state.end_time = datetime.now().isoformat()
            print(f"Errore durante il processing: {e}")
            return agent_state
    
    def save_agent_state(self, agent_state: AgentState, output_path: str):
        """Salva lo stato dell'agente su file"""
        try:
            # Converti in dizionario serializzabile
            state_dict = asdict(agent_state)
            
            # Rimuovi gli embedding che non sono serializzabili in JSON
            if state_dict.get("document") and state_dict["document"].get("embedding"):
                state_dict["document"]["embedding"] = None
            
            for chunk in state_dict.get("relevant_chunks", []):
                if chunk.get("embedding"):
                    chunk["embedding"] = None
            
            # Salva su file
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(state_dict, f, indent=2, ensure_ascii=False)
            
            print(f"Stato dell'agente salvato in {output_path}")
            
        except Exception as e:
            print(f"Errore nel salvare lo stato dell'agente: {e}")
    
    def format_agent_report(self, agent_state: AgentState) -> str:
        """Formatta un report leggibile dello stato dell'agente"""
        if not agent_state:
            return "Nessuno stato agente disponibile."
        
        # Estrai informazioni principali
        report = []
        report.append(f"# Rapporto Analisi Documento")
        report.append(f"ID Sessione: {agent_state.agent_id}")
        report.append(f"Stato: {agent_state.status}")
        
        if agent_state.document:
            report.append(f"\n## Documento")
            report.append(f"ID: {agent_state.document.id}")
            if agent_state.document.file_name:
                report.append(f"Nome file: {agent_state.document.file_name}")
            
            # Mostra un'anteprima del contenuto
            doc_preview = agent_state.document.content[:200] + "..." if len(agent_state.document.content) > 200 else agent_state.document.content
            report.append(f"\nAnteprima contenuto:\n```\n{doc_preview}\n```")
        
        # Decisione finale
        if agent_state.final_decision:
            report.append(f"\n## Decisione Finale")
            report.append(f"Tipo documento: **{agent_state.final_decision}**")
            report.append(f"Confidenza: {agent_state.confidence:.2f}")
        
        # Knowledge chunks
        if agent_state.relevant_chunks:
            report.append(f"\n## Fonti di Conoscenza Rilevanti")
            for i, chunk in enumerate(agent_state.relevant_chunks[:3]):  # Mostra solo i primi 3
                report.append(f"\n### Fonte {i+1} (Score: {chunk.relevance_score:.4f})")
                chunk_preview = chunk.content[:150] + "..." if len(chunk.content) > 150 else chunk.content
                report.append(f"```\n{chunk_preview}\n```")
            
            if len(agent_state.relevant_chunks) > 3:
                report.append(f"\n... e altre {len(agent_state.relevant_chunks) - 3} fonti")
        
        # Ragionamento
        if agent_state.reasoning_steps:
            report.append(f"\n## Processo di Ragionamento")
            for i, step in enumerate(agent_state.reasoning_steps):
                report.append(f"\n### Step {i+1}: {step.action}")
                report.append(step.thought)
                
                # Se disponibile, mostra l'output del ragionamento più dettagliato
                if step.action == "analyze_document" and isinstance(step.action_output, dict):
                    thoughts = step.action_output.get("thoughts")
                    observations = step.action_output.get("observations")
                    
                    if thoughts:
                        report.append(f"\nRagionamento dettagliato:")
                        report.append(f"```\n{thoughts}\n```")
                    
                    if observations:
                        report.append(f"\nOsservazioni:")
                        report.append(f"```\n{observations}\n```")
        
        # Calcola tempo totale
        if agent_state.start_time and agent_state.end_time:
            start = datetime.fromisoformat(agent_state.start_time)
            end = datetime.fromisoformat(agent_state.end_time)
            duration = (end - start).total_seconds()
            report.append(f"\n## Statistiche")
            report.append(f"Tempo di analisi: {duration:.2f} secondi")
        
        # Errore (se presente)
        if agent_state.error:
            report.append(f"\n## Errore")
            report.append(f"```\n{agent_state.error}\n```")
        
        return "\n".join(report)

# Funzione di test per l'AgenticRAG
def test_agentic_rag_system():
    # Inizializza il sistema RAG agentico
    rag_system = AgenticRAG()
    
    # Carica un documento di esempio
    example_docs = []
    directory = "anonymized" if os.path.exists("anonymized") else "documents"
    
    if os.path.exists(directory):
        files = os.listdir(directory)[:2]  # Prendi solo 2 file per test
        for file in files:
            file_path = os.path.join(directory, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    example_docs.append((file, content))
            except Exception as e:
                print(f"Errore nel leggere {file}: {e}")
    
    if not example_docs:
        # Usa un documento di esempio fittizio
        example_docs = [("example.txt", """
            Oggetto: Richiesta preventivo per fornitura materiale informatico
            
            Gentile fornitore,
            
            Con la presente siamo a richiedere un preventivo per l'acquisto del seguente materiale:
            
            - 10 laptop modello XYZ
            - 10 monitor 27"
            - 10 docking station universali
            
            Restiamo in attesa di un vostro riscontro.
            
            Cordiali saluti,
            Ufficio Acquisti
        """)]
    
    # Processa ogni documento
    results = []
    
    for file_name, content in example_docs:
        print(f"\n\n{'='*50}")
        print(f"Analisi documento: {file_name}")
        print(f"{'='*50}")
        
        # Process with the agentic system
        agent_state = rag_system.process_document(
            document_content=content,
            metadata={"file_name": file_name}
        )
        
        # Genera e stampa il report
        report = rag_system.format_agent_report(agent_state)
        print(report)
        
        # Salva lo stato dell'agente (opzionale)
        output_dir = "agent_outputs"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"agent_state_{file_name.replace('.', '_')}.json")
        rag_system.save_agent_state(agent_state, output_path)
        
        # Salva risultati per confronto
        results.append({
            "file": file_name,
            "classification": agent_state.final_decision,
            "confidence": agent_state.confidence,
            "reasoning_steps": len(agent_state.reasoning_steps)
        })
    
    return results

# Esegui il test (decommentare per eseguire)
# test_results = test_agentic_rag_system()

## Test del Sistema Agentico RAG

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import time

# Flag per l'esecuzione del test
run_agentic_test = False  # Cambiare in True per eseguire il test

if run_agentic_test:
    # Inizializza il sistema RAG agentico
    print("Inizializzazione del sistema RAG agentico...")
    rag_system = AgenticRAG()
    
    # Directory contenente i documenti da analizzare
    directory = "anonymized" if os.path.exists("anonymized") else "documents"
    
    if not os.path.exists(directory):
        print(f"Directory {directory} non trovata! Creazione di un documento di esempio...")
        os.makedirs("documents", exist_ok=True)
        with open("documents/example.txt", "w", encoding="utf-8") as f:
            f.write("""
            Oggetto: Richiesta preventivo per fornitura materiale informatico
            
            Gentile fornitore,
            
            Con la presente siamo a richiedere un preventivo per l'acquisto del seguente materiale:
            
            - 10 laptop modello XYZ
            - 10 monitor 27"
            - 10 docking station universali
            
            Restiamo in attesa di un vostro riscontro.
            
            Cordiali saluti,
            Ufficio Acquisti
            """)
        
        directory = "documents"

    # Carica i documenti di test (limita a 3 per brevità)
    files = os.listdir(directory)[:3]
    
    # Prepara strutture dati per i risultati
    results = []
    agent_states = []
    
    # Analizza ogni documento
    for file in files:
        file_path = os.path.join(directory, file)
        print(f"\nAnalisi del documento: {file}")
        
        try:
            # Leggi contenuto del documento
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
            
            # Registra il tempo di inizio
            start_time = time.time()
            
            # Processa con il sistema agentico
            agent_state = rag_system.process_document(
                document_content=content,
                metadata={"file_name": file, "file_path": file_path}
            )
            
            # Calcola tempo di esecuzione
            execution_time = time.time() - start_time
            
            # Salva risultati
            agent_states.append(agent_state)
            results.append({
                "file": file,
                "classification": agent_state.final_decision,
                "confidence": agent_state.confidence,
                "execution_time": execution_time,
                "steps": len(agent_state.reasoning_steps),
                "status": agent_state.status,
                "num_chunks": len(agent_state.relevant_chunks)
            })
            
            # Visualizza report del documento corrente
            display(Markdown(rag_system.format_agent_report(agent_state)))
            
        except Exception as e:
            print(f"Errore nell'analisi di {file}: {e}")
    
    # Crea dataframe con risultati
    df_results = pd.DataFrame(results)
    
    # Visualizza tabella risultati
    print("\n=== RIEPILOGO RISULTATI ===")
    display(df_results)
    
    # Visualizza grafici dei risultati
    plt.figure(figsize=(14, 6))
    
    # Grafico classificazioni
    plt.subplot(1, 2, 1)
    if not df_results.empty:
        classification_counts = df_results['classification'].value_counts()
        classification_counts.plot.bar(color='skyblue')
        plt.title('Classificazioni Documenti')
        plt.xlabel('Tipo Documento')
        plt.ylabel('Numero Documenti')
        plt.xticks(rotation=45)
    
    # Grafico confidenza
    plt.subplot(1, 2, 2)
    if not df_results.empty:
        plt.bar(df_results['file'], df_results['confidence'], color='lightgreen')
        plt.title('Confidenza delle Classificazioni')
        plt.xlabel('Documento')
        plt.ylabel('Confidenza')
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Visualizza dettagli performance
    if not df_results.empty:
        print("\n=== STATISTICHE PERFORMANCE ===")
        print(f"Tempo medio di esecuzione: {df_results['execution_time'].mean():.2f} secondi")
        print(f"Numero medio di step di ragionamento: {df_results['steps'].mean():.1f}")
        print(f"Numero medio di chunk rilevanti: {df_results['num_chunks'].mean():.1f}")
else:
    print("Test non eseguito. Per eseguire il test, imposta 'run_agentic_test = True'.")

## Confronto tra RAG Standard e RAG Agentico

### RAG Standard vs RAG Agentico

Il **RAG standard** (Retrieval-Augmented Generation) arricchisce i prompt con informazioni rilevanti recuperate da una knowledge base. Funziona in questo modo:

1. Si genera l'embedding del documento
2. Si recuperano chunks simili dalla knowledge base
3. Si inseriscono i chunks nel prompt
4. Si ottiene una risposta che considera sia il documento che le informazioni aggiuntive

Il **RAG agentico** invece estende questo approccio aggiungendo capacità di ragionamento autonomo, pianificazione e decisione. I componenti principali sono:

1. **Sistema di ragionamento multi-step**: l'agente analizza il documento in più fasi, registrando pensieri e osservazioni
2. **Tracciamento dello stato**: l'agente mantiene uno stato interno che evolve durante l'analisi
3. **Pianificazione e decisioni**: l'agente decide autonomamente cosa fare con l'informazione
4. **Auto-valutazione**: l'agente valuta la propria confidenza nelle decisioni prese
5. **Memoria**: l'agente tiene traccia del reasoning e delle decisioni precedenti

### Vantaggi del RAG Agentico

- **Trasparenza**: l'intero processo decisionale è documentato e ispezionabile
- **Confidenza quantificata**: ogni decisione è accompagnata da una misura di confidenza
- **Flessibilità**: può adattarsi a diversi tipi di documenti e casi d'uso
- **Decision-making avanzato**: ragiona sulle informazioni invece di limitarsi a classificare
- **Persistenza dello stato**: il processo può essere salvato, analizzato e ripreso

### Limitazioni del RAG Agentico

- **Complessità**: richiede più chiamate API e logica applicativa
- **Costo computazionale**: richiede più risorse rispetto al RAG standard
- **Latenza**: il processo multi-step può richiedere più tempo
- **Overhead di implementazione**: richiede più codice e manutenzione

### Casi d'uso ideali per il RAG Agentico

- Analisi di documenti complessi
- Processi decisionali che richiedono più passaggi
- Situazioni in cui è essenziale la trasparenza del ragionamento
- Attività che richiedono un alto livello di confidenza nelle decisioni
- Workflow che potrebbero richiedere azioni differenti in base al contenuto

## Integrazione di DOCUMENT_TYPE_PROMPT con il Sistema Agentico RAG

In [ ]:
import os
import openai
from dotenv import load_dotenv
import pickle
import numpy as np
import json
from datetime import datetime
import time

# Carica variabili d'ambiente
load_dotenv()

# DOCUMENT_TYPE_PROMPT originale
DOCUMENT_TYPE_PROMPT = """
Ti fornirò un documento aziendale.
 
Il tuo compito è determinare il tipo di documento, scegliendo tra i seguenti: 
- Mail
- Nota di credito
- Ordine di acquisto
- Contratto
- Altro
 
Devi basarti solo sul contenuto del documento fornito.
 
Ora incollerò il contenuto del documento tra tripli apici. Rispondi semplicemente con il tipo, nulla di più.
 
Documento:
'''
{documento}
'''
"""

# Versione agentica del DOCUMENT_TYPE_PROMPT
AGENTIC_DOCUMENT_TYPE_PROMPT = """
Ti fornirò un documento aziendale.

Il tuo compito è determinare il tipo di documento, scegliendo tra i seguenti: 
- Mail
- Nota di credito
- Ordine di acquisto
- Contratto
- Altro

Devi utilizzare un processo di ragionamento strutturato:
1. Analizza attentamente il contenuto del documento
2. Considera le informazioni rilevanti dalla knowledge base
3. Identifica pattern e caratteristiche chiave per la classificazione
4. Fornisci un ragionamento dettagliato per la tua classificazione
5. Esprimi un livello di confidenza nella tua decisione (da 0 a 1)

Rispondi in formato JSON con:
- thoughts: il tuo ragionamento dettagliato
- classification: il tipo di documento
- confidence: un valore numerico tra 0 e 1
- evidence: elementi o frasi specifiche che hanno guidato la tua decisione

Ecco informazioni rilevanti dalla knowledge base:

{context}

Ora, analizza il documento tra tripli apici:
'''
{documento}
'''
"""

class IntegratedAgenticDocumentClassifier:
    """Integratore tra il DOCUMENT_TYPE_PROMPT originale e il sistema RAG agentico"""
    
    def __init__(self, knowledge_base_path="chunks_with_embeddings.pkl"):
        """Inizializza il classificatore con la knowledge base"""
        self.rag_system = AgenticRAG(knowledge_base_path)
        
        # Configurazione API
        self.api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.api_version = os.getenv('AZURE_OPENAI_API_VERSION')
        self.deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT')
        
        self.client = openai.AzureOpenAI(
            api_key=self.api_key,
            azure_endpoint=self.endpoint,
            api_version=self.api_version
        )
    
    def _enhance_prompt_with_context(self, document_text):
        """Arricchisce il prompt con contesto rilevante"""
        # Crea documento e trova chunks rilevanti
        doc = Document(
            id=f"doc_{datetime.now().timestamp()}",
            content=document_text
        )
        
        # Genera embedding
        doc.embedding = self.rag_system.create_embedding(doc.content[:1500])
        if doc.embedding is None:
            return DOCUMENT_TYPE_PROMPT.format(documento=document_text), []
        
        # Recupera chunk rilevanti
        relevant_chunks = self.rag_system.retrieve_relevant_chunks(doc, top_k=3)
        
        # Se non ci sono chunk rilevanti, usa il prompt standard
        if not relevant_chunks:
            return DOCUMENT_TYPE_PROMPT.format(documento=document_text), []
        
        # Componi il contesto
        context_parts = []
        for i, chunk in enumerate(relevant_chunks):
            context_parts.append(
                f"[FONTE {i+1}] (Rilevanza: {chunk.relevance_score:.4f})\n{chunk.content}"
            )
        
        context = "\n\n".join(context_parts)
        
        # Crea prompt arricchito
        enhanced_prompt = AGENTIC_DOCUMENT_TYPE_PROMPT.format(
            context=context,
            documento=document_text
        )
        
        return enhanced_prompt, relevant_chunks
    
    def classify_standard(self, document_text):
        """Classifica usando il DOCUMENT_TYPE_PROMPT originale"""
        prompt = DOCUMENT_TYPE_PROMPT.format(documento=document_text)
        
        response = self.client.chat.completions.create(
            model=self.deployment_name,
            messages=[
                {"role": "system", "content": "Sei un analizzatore di documenti aziendali."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=50
        )
        
        return response.choices[0].message.content.strip()
    
    def classify_agentic(self, document_text):
        """Classifica usando l'approccio agentico"""
        # Processa il documento con il sistema agentico
        agent_state = self.rag_system.process_document(document_text)
        
        return {
            "classification": agent_state.final_decision,
            "confidence": agent_state.confidence,
            "reasoning": [step.thought for step in agent_state.reasoning_steps],
            "num_chunks": len(agent_state.relevant_chunks)
        }
    
    def classify_hybrid(self, document_text):
        """Approccio ibrido che usa il prompt arricchito ma non l'intero sistema agentico"""
        enhanced_prompt, relevant_chunks = self._enhance_prompt_with_context(document_text)
        
        try:
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=[
                    {"role": "system", "content": "Sei un analizzatore di documenti aziendali."},
                    {"role": "user", "content": enhanced_prompt}
                ],
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            return {
                "classification": result.get("classification", "Altro"),
                "confidence": result.get("confidence", 0.5),
                "thoughts": result.get("thoughts", ""),
                "evidence": result.get("evidence", ""),
                "num_chunks": len(relevant_chunks)
            }
        except Exception as e:
            print(f"Errore nella classificazione ibrida: {e}")
            # Fallback a classificazione standard
            return {
                "classification": self.classify_standard(document_text),
                "confidence": 0.5,
                "thoughts": "Errore nell'elaborazione agentica, usando classificazione standard.",
                "evidence": "",
                "num_chunks": len(relevant_chunks)
            }
    
    def compare_approaches(self, document_text):
        """Confronta i tre approcci diversi"""
        results = {}
        
        print("Confronto dei tre approcci di classificazione...")
        
        # Standard (originale)
        start = time.time()
        standard_result = self.classify_standard(document_text)
        standard_time = time.time() - start
        results["standard"] = {
            "classification": standard_result,
            "time": standard_time,
            "approach": "Standard (DOCUMENT_TYPE_PROMPT originale)"
        }
        print(f"Classificazione standard: {standard_result} ({standard_time:.2f}s)")
        
        # Hybrid (prompt arricchito)
        start = time.time()
        hybrid_result = self.classify_hybrid(document_text)
        hybrid_time = time.time() - start
        results["hybrid"] = {
            **hybrid_result,
            "time": hybrid_time,
            "approach": "Hybrid (DOCUMENT_TYPE_PROMPT con RAG)"
        }
        print(f"Classificazione ibrida: {hybrid_result['classification']} (confidenza: {hybrid_result['confidence']:.2f}, {hybrid_time:.2f}s)")
        
        # Agentic (completo)
        start = time.time()
        agentic_result = self.classify_agentic(document_text)
        agentic_time = time.time() - start
        results["agentic"] = {
            **agentic_result,
            "time": agentic_time,
            "approach": "Agentic (Sistema RAG completo)"
        }
        print(f"Classificazione agentica: {agentic_result['classification']} (confidenza: {agentic_result['confidence']:.2f}, {agentic_time:.2f}s)")
        
        return results

# Funzione per dimostrare l'integrazione
def test_integrated_approach(run_demo=False):
    if not run_demo:
        print("Demo non eseguita. Imposta run_demo=True per eseguire.")
        return
    
    # Inizializza il classificatore integrato
    classifier = IntegratedAgenticDocumentClassifier()
    
    # Carica un documento di test
    document_text = ""
    directory = "anonymized" if os.path.exists("anonymized") else "documents"
    
    if os.path.exists(directory) and os.listdir(directory):
        # Prendi il primo file
        file_path = os.path.join(directory, os.listdir(directory)[0])
        with open(file_path, "r", encoding="utf-8") as f:
            document_text = f.read()
    else:
        # Usa un documento di esempio
        document_text = """
        Oggetto: Richiesta preventivo per fornitura materiale informatico
        
        Gentile fornitore,
        
        Con la presente siamo a richiedere un preventivo per l'acquisto del seguente materiale:
        
        - 10 laptop modello XYZ
        - 10 monitor 27"
        - 10 docking station universali
        
        Restiamo in attesa di un vostro riscontro entro il 15/07/2025.
        
        Cordiali saluti,
        Ufficio Acquisti
        """
    
    # Confronta i tre approcci
    print("\n" + "="*50)
    print("CONFRONTO TRA I TRE APPROCCI")
    print("="*50)
    
    results = classifier.compare_approaches(document_text)
    
    # Visualizza riepilogo
    print("\n" + "-"*50)
    print("RIEPILOGO RISULTATI")
    print("-"*50)
    
    print(f"Standard: {results['standard']['classification']} ({results['standard']['time']:.2f}s)")
    print(f"Hybrid: {results['hybrid']['classification']} (conf: {results['hybrid']['confidence']:.2f}, {results['hybrid']['time']:.2f}s)")
    print(f"Agentic: {results['agentic']['classification']} (conf: {results['agentic']['confidence']:.2f}, {results['agentic']['time']:.2f}s)")
    
    # Visualizza dettagli del ragionamento agentico
    if results['agentic']['reasoning']:
        print("\n" + "-"*50)
        print("RAGIONAMENTO AGENTICO")
        print("-"*50)
        
        for i, reasoning in enumerate(results['agentic']['reasoning']):
            print(f"Step {i+1}: {reasoning[:150]}...")
    
    return results

# Esegui il test (decommentare per eseguire)
# demo_results = test_integrated_approach(run_demo=True)